In [2]:
from base_regression import *

In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_regression
from sklearn.metrics import *


In [4]:
X, y = make_regression(1000, n_features=30, n_informative=5)

In [5]:
df = pd.read_csv("data/trees.csv")

In [6]:
df.head()

,Girth,Height,Volume
0,8.3,70.0,10.3
1,8.6,65.0,10.3
2,8.8,63.0,10.2
3,10.5,72.0,16.4
4,10.7,81.0,18.8


In [9]:
X = df[['Girth', 'Height']]
y = np.array(df[['Volume']]).flatten()

In [10]:
current_model = BMARS(X, 2)
eval_pipeline(current_model.construct_pipeline(False), X, y)

Result: -2.2217477271935316e+25


-2.2217477271935316e+25

In [11]:
alpha, model1 = mh_iter(X, y, current_model)

action: birth


TypeError: unhashable type: 'slice'

In [6]:
%%time
# run for 10 iters...
iters = 50

for _ in range(iters):
    try:
        alpha, next_model = mh_iter(X, y, current_model, False)
        current_model = BMARS(**next_model)   
    except:
        print("\n---\n")
        pass
    

action: birth
action: birth
action: change
action: birth
action: change
action: change
action: birth
action: birth
action: change
action: change
action: birth
action: birth
action: change
action: change
action: birth
action: birth
action: change
action: change
action: death
action: change
action: death
action: death
action: birth
action: birth
action: birth
action: death
action: change
action: birth
action: change
action: birth
action: death
action: change
action: death
action: death
action: birth
action: birth
action: death
action: change
action: change
action: change
action: death
action: death
action: change
action: change
action: birth
action: birth
action: birth
action: change
action: change
action: birth
Wall time: 12min 39s


In [7]:
eval_pipeline(current_model.construct_pipeline(False), X, y)

Result: 0.999999463456924


0.99999946345692403

In [8]:
current_model.export()

{'X': array([[ 0.01901448,  2.05185057,  0.15917775, -0.69659869, -0.92705932,
          0.6606366 , -0.19621994,  0.10774462, -1.40490027, -0.33064417,
         -0.10514226,  0.59698639, -0.05882021,  0.47215394, -1.68397143,
         -0.44020751,  1.88561662, -0.28747344,  1.32389713, -0.70790102,
          0.41457444, -0.83584958,  1.03273505, -0.66021199,  0.82123069,
         -1.56448551, -0.13069419,  1.70008861, -0.30117336, -1.12802154]]),
 'basis': [[14, 29],
  [5, 12],
  [1, 7],
  [14, 23],
  [2, 20],
  [0, 4],
  [0, 23],
  [2, 7],
  [27, 21],
  [7, 15]],
 'interaction': 2,
 'params': [{'knots': array([-1.90657694,  0.63927268]), 'signs': [1, 1]},
  {'knots': array([-0.69768139,  1.52801491]), 'signs': [-1, 1]},
  {'knots': array([-1.44637715,  0.84477687]), 'signs': [-1, 1]},
  {'knots': array([ 1.02285599,  0.16142696]), 'signs': [1, -1]},
  {'knots': array([-0.12716787,  1.64778461]), 'signs': [1, -1]},
  {'knots': array([ 0.4245224 ,  0.35495156]), 'signs': [1, 1]},
  {'k